In [17]:
#다음 뉴스 기사 추출
import re
import requests
from bs4 import BeautifulSoup

res = requests.get('https://news.daum.net/')
soup = BeautifulSoup(res.content, 'html.parser')
soup = soup.select('#kakaoContent')[0]

extracts = [dict(
    title = re.sub('\s+', ' ', a.get_text().replace('\n', '')),
    link = a['href']
) for a in soup.select('a.link_txt')]

#extracts

In [18]:
# 네이버 실시간 검색어 추출
import requests
from bs4 import BeautifulSoup

res = requests.get('https://www.naver.com/')
soup = BeautifulSoup(res.content, 'html.parser')

In [20]:
#copy cURL
import requests

headers = {
    'authority': 'apis.naver.com',
    'pragma': 'no-cache',
    'cache-control': 'no-cache',
    'accept': 'application/json, text/plain, */*',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
    'origin': 'https://www.naver.com',
    'sec-fetch-site': 'same-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.naver.com/',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

params = (
    ('frm', 'main'),
    ('ag', '30s'),
    ('gr', '2'),
    ('ma', '-2'),
    ('si', '0'),
    ('en', '-2'),
    ('sp', '-2'),
)

response = requests.get('https://apis.naver.com/mobile_main/srchrank/srchrank', headers=headers, params=params)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://apis.naver.com/mobile_main/srchrank/srchrank?frm=main&ag=30s&gr=2&ma=-2&si=0&en=-2&sp=-2', headers=headers)

response.json()

{'ts': '2020-05-14T17:39:00+0900',
 'sm': 'ag30sgrpmamsi0enmspm',
 'rop': [{'ag': '30s'},
  {'gr': '02'},
  {'ma': '-2'},
  {'si': '00'},
  {'en': '-2'},
  {'sp': '-2'}],
 'data': [{'rank': 1, 'keyword': 'sk케미칼', 'keyword_synonyms': []},
  {'rank': 2,
   'keyword': '한국파스퇴르연구소',
   'keyword_synonyms': ['한국파스퇴르연구소 주식', '큐리언트', '한국파스퇴르', '파스퇴르']},
  {'rank': 3, 'keyword': 'a3 스틸얼라이브', 'keyword_synonyms': ['a3']},
  {'rank': 4, 'keyword': '혜민동물병원', 'keyword_synonyms': []},
  {'rank': 5, 'keyword': '스승의날 문구', 'keyword_synonyms': ['스승의날 편지']},
  {'rank': 6, 'keyword': '괴질', 'keyword_synonyms': []},
  {'rank': 7, 'keyword': '샘표식품', 'keyword_synonyms': ['샘표']},
  {'rank': 8, 'keyword': 'SK 바이오랜드', 'keyword_synonyms': []},
  {'rank': 9, 'keyword': '태풍', 'keyword_synonyms': []},
  {'rank': 10, 'keyword': '유연상', 'keyword_synonyms': []},
  {'rank': 11, 'keyword': '김무성', 'keyword_synonyms': []},
  {'rank': 12,
   'keyword': '나파모스타트 관련주',
   'keyword_synonyms': ['한미약품', '제일약품', '나파모스타트', '뉴지랩']},
  

In [29]:
ratings = {
     '이숙번': {'1917': 5, '엽문4': 2, '라라랜드': 3, '주디': 5},
     '강두루': {'1917': 4, '라라랜드': 3, '신과나': 5},
     '이고잉': {'라라랜드': 4, '엽문4': 4, '주디': 1},
     '정원혁': {'엽문':3, '신과나': 5, '1917': 4, '주디': 2}
}

keys = [list(v.keys()) for _, v in ratings.items()] #_: k 받아서 안쓰고 싶을 때

#print(sum(keys, [])) #list들의 합 가능(이중 리스트라 이렇게 합쳐야함)
keys = set(sum(keys, [])) # 중복 제거

movies = []
for key in keys:
    movies.append({
        '영화': key,
        '평점': [v.get(key, 0) for k, v in ratings.items()] #0은 Default 값, v의 값이 없을 수도 있으니까 get으로 가져오기
    })

movies

[{'영화': '1917', '평점': [5, 4, 0, 4]},
 {'영화': '라라랜드', '평점': [3, 3, 4, 0]},
 {'영화': '엽문4', '평점': [2, 0, 4, 0]},
 {'영화': '엽문', '평점': [0, 0, 0, 3]},
 {'영화': '주디', '평점': [5, 0, 1, 2]},
 {'영화': '신과나', '평점': [0, 5, 0, 5]}]

In [30]:
ratings = {
     '이숙번': {'1917': 5, '엽문4': 2, '라라랜드': 3, '주디': 5},
     '강두루': {'1917': 4, '라라랜드': 3, '신과나': 5},
     '이고잉': {'라라랜드': 4, '엽문4': 4, '주디': 1},
     '정원혁': {'엽문':3, '신과나': 5, '1917': 4, '주디': 2}
}

import math

# 이숙번과 취향이 가장 비슷한 사람을 찾아라.
def diff(user1, user2):
    # user1.keys() -> ['1917', '엽문', '라라랜드', '주디']
    # user2.keys() -> ['1917', '라라랜드', '신과나']
    # 교집합 keys -> ['1917', '라라랜드']
    keys = set(user1.keys()).intersection(set(user2.keys()))
    
    # [(5 - 4) ** 2, (3 - 3) ** 2]
    differs = sum([(user1[k] - user2[k]) ** 2 for k in keys])

    return math.sqrt(differs)

print(diff(ratings['이숙번'], ratings['강두루']))
print(diff(ratings['이숙번'], ratings['이고잉']))
print(diff(ratings['이숙번'], ratings['정원혁']))

1.0
4.58257569495584
3.1622776601683795
